## Điều chuẩn Lasso

Điều chuẩn gồm việc thêm một penalty vào các tham số khác nhau của mô hình học máy để giảm bớt bậc tự do của mô hình và tránh overfitting. Điều chuẩn mô hình tuyến tính áp dụng penalty cho các hệ số nhân từng yếu tố dự đoán. Điều chuẩn Lasso hoặc l1 có thuộc tính thu nhỏ một số hệ số về 0 nên có thể loại bỏ các đặc trưng đó khỏi mô hình.

Chúng ta sẽ minh họa cách lựa chọn đặc trưng sử dụng điều chuẩn Lasso trong bài toán hồi quy và phân loại.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

## Hồi quy

In [2]:
# load tập dữ liệu

data = pd.read_csv('lab6-13_dataset/house-price/houseprice.csv')
data.shape

(1460, 81)

In [3]:
# Trên thực tế, nên thực hiện lựa chọn đặc trưng sau khi tiền xử lý dữ liệu,
# tất cả các biến hạng mục được mà hóa thành số,
# và rồi chúng ta có thể đánh giá tính tất định của các mục tiêu

# để đơn giản, chúng ta sẽ chỉ sử dụng biến dạng số
# lựa chọn các cột số:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [4]:
# chia thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 37), (438, 37))

In [5]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [6]:
# các đặc trưng trong tập dữ liệu giá nhà ở 
# các thang đo khác nhau nên nó giúp hồi quy co giãn
# các đặc trưng

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

### Lựa chọn các hệ số với Lasso

In [8]:
## Yêu cầu 1
# ở đây, chúng ta huấn luyện hồi quy tuyến tính Lasso và lựa chọn
# các đặc trưng khác 0 trên 1 dòng.

# nhớ rằng đối tượng hồi quy tuyến tính từ sklearn
# không cho phép điều chuẩn. Nên nếu chúng ta muốn tạo
# hồi quy tuyến tính được điều chuẩn thì cần đặc biệt import "Lasso"
 
# alpha là lượng phạt nên chúng ta đặt alpha lớn
# để buộc thuật toán thu nhỏ các hệ số

## VIẾT CODE Ở ĐÂY:
sel_ = SelectFromModel(Lasso(alpha=100, random_state=10))
sel_.fit(scaler.transform(X_train), y_train)

SelectFromModel(estimator=Lasso(alpha=100, random_state=10))

<details><summary> Gợi ý </summary>

[SelectFromModel()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html)

</details>

In [9]:
sel_.get_support()

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True])

In [11]:
## Yêu cầu 2
# tạo một list với các đặc trưng đã chọn và in ra kết quả
selected_feat = X_train.columns[(sel_.get_support())]

print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
## VIẾT CODE Ở ĐÂY:
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

total features: 37
selected features: 33
features with coefficients shrank to zero: 4


<details><summary> Gợi ý </summary>

Using ```estimator_.coef_``` attribute

</details>

Chúng ta có thể sử dụng điều chuẩn Lasso với cả hồi quy tuyến tính và logistic để loại bỏ các đặc trưng không quan trọng khỏi tập dữ liệu.


Hãy nhớ rằng việc tăng lượng phạt cũng sẽ gia tăng số lượng đặc trưng bị loại. Do đó, chúng ta cần phải theo dõi chất lượng cuối cùng của mô hình để đảm bảo không đặt lượng phạt quá cao khiến nhiều đặc trưng bị loại bỏ hoặc quá thấp khiến các đặc trưng vô dụng vẫn được giữ lại. 
